In [2]:
source('detlearn_analysis.R')
library('rjags')
library('coda')
library('parallel')
library('plyr')
library('ggplot2')
library('gridExtra')

Loading required package: coda
Linked to JAGS 3.4.0
Loaded modules: basemod,bugs


In [2]:
parallelize = T #set to false to debug
completedModelPath = 'mcmc/results/'

Build the directory structure...

In [3]:
paths = buildDirectoryStructure('results/')
paths['completedModelPath'] = completedModelPath

Check for model completion...

In [4]:
modelsComplete = checkForModelCompletion(completedModelPath, outfile=NULL)
if (nrow(subset(modelsComplete, !changePresence)) > 0){stop(p('One or more models is not availabe in',completedModelPath))}

# Extract Model Results

### Split Half Models

In [5]:
lcFileList = paste(completedModelPath, list.files(completedModelPath)[grep('linkingChange_Model.*\\.RData',list.files(completedModelPath))], sep='')
lcInputs.df = data.frame(file = lcFileList, paths['tablePath'], stringsAsFactors=F)
lcInputs = split(lcInputs.df, lcInputs.df$file)

In [6]:
linkingChangeWindowAnalyses = mclapply2(lcInputs, function(lcInput){
    processModelReturnData(lcInput$file, overwrite=T, lcInput$tablePath)
}, parallelize=F)

### Sliding Window Models

In [7]:
swFileList = paste(completedModelPath, list.files(completedModelPath)[grep('linking_Model.*\\.RData',list.files(completedModelPath))], sep='')
swInputs.df = data.frame(file = swFileList, paths['tablePath'], stringsAsFactors=F)
swInputs = split(swInputs.df, swInputs.df$file)

In [8]:
slidingWindowAnalyses = mclapply2(swInputs, function(swInput){
    processModelReturnData(swInput$file, overwrite=T, swInput$tablePath)
})

# Convergence Statistics

In [ ]:
convergenceTable = do.call('rbind',lapply(c('all_LN','singulars_LN','all_FN','singulars_LN','all_standard','singulars_standard','all_standard','none_standard'), function(dataPrep){
    mr = getModelResults(paths[['tablePath']],'linking', dataPrep)
    data.frame(dataPrep, numConverged = length(which(!is.na(mr$nu.child_mean))), numTotal = length(mr$nu.child_mean))
}))
convergenceTable$propFailed = convergenceTable$numConverged / convergenceTable$numTotal
print(convergenceTable)

# Split Half Plots

In [3]:
shp = makeSplitHalfPlots(paths, 'singulars_LN')
print(shp[['splitHalfTimeCourse']])

ERROR: Error in list.files(summaryDir): object 'paths' not found


ERROR: Error in print(shp[["splitHalfTimeCourse"]]): object 'shp' not found


In [ ]:
print(shp[['violinPlot']])

In [ ]:
shc = splitHalfComparison(paths)
print(shc[['phase1NuComparison']])

In [ ]:
print(shc[['phase2NuComparison']])

In [ ]:
print(shc[['nuDifference']])

# Sliding Window Plots

In [ ]:
swp = makeSlidingWindowPlots(paths, 'singulars_LN', shp[['childrenToKeep']])
print(swp)

# Correlations

In [ ]:
source('detlearn_analysis.R')
cors = getCorrelations(paths)
print(cors)